In [1]:
cd ..

/Users/shuai/Marshal/Mine/analysis/CS231n/assignment2


In [2]:
# As usual, a bit of setup

import numpy as np
import matplotlib.pyplot as plt
from cs231n.classifiers.multi_cnn import *
from cs231n.data_utils import get_CIFAR10_data
from cs231n.gradient_check import eval_numerical_gradient_array, eval_numerical_gradient
from cs231n.layers import *
from cs231n.fast_layers import *
from cs231n.solver import Solver

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [3]:
# Load the (preprocessed) CIFAR10 data.

data = get_CIFAR10_data()
for k, v in data.iteritems():
  print '%s: ' % k, v.shape

X_val:  (1000, 3, 32, 32)
X_train:  (49000, 3, 32, 32)
X_test:  (1000, 3, 32, 32)
y_val:  (1000,)
y_train:  (49000,)
y_test:  (1000,)


### This sheet just uses random 2000 images in train set and checking parameters

In [4]:
np.random.seed(seed=123)

train_num = 1000
val_num = 100
train_indices = np.random.choice(data['X_train'].shape[0], train_num)
val_indices = np.random.choice(data['X_val'].shape[0], val_num)

X_train = data['X_train'][train_indices]
y_train = data['y_train'][train_indices]
X_val = data['X_val'][val_indices]
y_val = data['y_val'][val_indices]

print X_train.shape
print y_train.shape
print X_val.shape
print y_val.shape

(1000, 3, 32, 32)
(1000,)
(100, 3, 32, 32)
(100,)


Here only try to use [conv -> relu -> conv -> relu -> pool] x 2 -> [affine] x 3 -> softmax

In [5]:
filter_times = 2
model = MultiLayersCNN(num_filters=[32,]*filter_times, 
                       filter_sizes=[3,]*filter_times, 
                       hidden_dims=[100, ])

N = 50
X = np.random.randn(N, 3, 32, 32)
y = np.random.randint(10, size=N)

loss, grads = model.loss(X, y)
print 'Initial loss (no regularization): ', loss

#model.reg = 0.5
#loss, _ = model.loss(X, y)
#print 'Initial loss (with regularization): ', loss

Initial loss (no regularization):  2.30258509066


In [6]:
sorted(model.params.keys())

['W1', 'W2', 'W3', 'W4', 'b1', 'b2', 'b3', 'b4']

In [7]:
sorted(model.dout.keys())

[1, 2, 3, 4]

In [8]:
for k, v in model.out.iteritems():
    print k, v.shape

1 (50, 32, 32, 32)
2 (50, 32, 16, 16)
3 (50, 100)
4 (50, 10)


In [9]:
for k, v in model.dout.iteritems():
    print k, v.shape

1 (50, 3, 32, 32)
2 (50, 32, 32, 32)
3 (50, 32, 16, 16)
4 (50, 100)


In [30]:
num_inputs = 2
input_dim = (3, 16, 16)
reg = 0.0
num_classes = 10
X = np.random.randn(num_inputs, *input_dim)
y = np.random.randint(num_classes, size=num_inputs)

filter_times = 2
model = MultiLayersCNN(num_filters=[5,]*filter_times, 
                       filter_sizes=[3,]*filter_times,
                       hidden_dims=[10,], input_dim=input_dim,
                       dtype=np.float64, reg=reg, weight_scale=5e-2)

loss, grads = model.loss(X, y)
print "Model loss done"

for param_name in sorted(grads):
    f = lambda _: model.loss(X, y)[0]
    param_grad_num = eval_numerical_gradient(f, model.params[param_name], verbose=False, h=1e-6)
    e = rel_error(param_grad_num, grads[param_name])
    print '%s max relative error: %e' % (param_name, rel_error(param_grad_num, grads[param_name]))

Model loss done
W1 max relative error: 2.079225e-06
W2 max relative error: 1.556707e-06
W3 max relative error: 9.518873e-05
W4 max relative error: 2.614731e-07
b1 max relative error: 8.955939e-08
b2 max relative error: 1.444426e-08
b3 max relative error: 1.986835e-09
b4 max relative error: 1.501366e-09
